<a href="https://colab.research.google.com/github/julietamponti/tp2Datos-2C2021-/blob/main/4)_Consigna_35_terminado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***¿Cuál es el mínimo que ha durado desde su registro un usuario bloqueado en la plataforma?***

Realizo la instalación de todo lo que utilizare en el desarrollo de la consulta.

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 54.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=2f357144c4c263071725cdea85380245e557e4f432bb8eba278ac472c91b225d
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hi

In [ ]:
!pip install pyarrow

Importo las librerias.

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd 

In [ ]:
import pyarrow

Creo el SparkSession.

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Accedo a mi google drive para acceder al archivo.

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')
sqlContext = SQLContext(sc)

Mounted at /content/gdrive


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Lo pase a .parquet desde el local al archivo, porque desde colab no me dejaba, me saltaba un error con el drive.


Leo el archivo.

In [ ]:
logs = sqlContext.read.parquet('gdrive/My Drive/TP2 - Datos/logs.parquet')

In [ ]:
rddLogs = logs.rdd

Limpio el rdd.

In [ ]:
rddLogs = rddLogs.filter(lambda x: x.contributor_username != None and x.title != None)

Filtro por bloqueados.

In [ ]:
rddBloq = rddLogs.filter(lambda x: x.logtype == 'block')

Averiguo cuales fueron los usuarios bloqueados y en que fecha. Me quedo con ese formato de RDD.

In [ ]:
rddBlock = rddBloq.map(lambda x: (x.title.split(':')[1],pd.to_datetime(x.timestamp)))

Libero RAM.

In [ ]:
del rddBloq

In [ ]:
rddBlock.take(2)

[('212.55.30.157', Timestamp('2004-12-23 08:28:32+0000', tz='UTC')),
 ('201.241.228.15', Timestamp('2004-12-27 17:54:50+0000', tz='UTC'))]

Me quedo con la fecha del primer bloqueo del usuario.

In [ ]:
rddBlock = rddBlock.reduceByKey(lambda a,b: a if a < b else b)

In [ ]:
rddBlock.take(2)

[('201.255.50.32', Timestamp('2004-12-27 17:57:12+0000', tz='UTC')),
 ('81.192.30.42', Timestamp('2004-12-29 14:12:24+0000', tz='UTC'))]

Filtro por registro de usuarios (durante todo el desarrollo trabajo con los usuarios ***registrados***).

In [ ]:
rddNewUser = rddLogs.filter(lambda x: x.logtype == 'newusers')

Libero RAM.

In [ ]:
del rddLogs

Me quedo con un RDD que tiene el username del usuario creado y la fecha en que se creo el usuario.

In [ ]:
rddUser =  rddNewUser.map(lambda x: (x.contributor_username,pd.to_datetime(x.timestamp)))

Libero RAM.

In [ ]:
del rddNewUser

In [ ]:
rddUser.take(2)

[('Yliana', Timestamp('2005-09-07 22:33:56+0000', tz='UTC')),
 ('BrokenSegue', Timestamp('2005-09-07 22:50:58+0000', tz='UTC'))]

Me quedo con la primer fecha de registro del usuario.

In [ ]:
rddUser = rddUser.reduceByKey(lambda a,b: a if a < b else b)

In [ ]:
rddUser.take(2)

[('Luis forero', Timestamp('2005-09-15 15:56:52+0000', tz='UTC')),
 ('Jcaraya~eswiki', Timestamp('2005-09-17 17:13:59+0000', tz='UTC'))]

Realizo un join para tener (username, (fecha de alta, fecha de bloqueo)).

In [ ]:
rddJoin = rddUser.join(rddBlock)

In [ ]:
rddJoin.take(2)

[('Kakabron',
  (Timestamp('2010-06-11 21:27:19+0000', tz='UTC'),
   Timestamp('2020-10-03 06:00:37+0000', tz='UTC'))),
 ('Proyectoewa',
  (Timestamp('2009-12-13 20:18:23+0000', tz='UTC'),
   Timestamp('2009-12-13 20:34:07+0000', tz='UTC')))]

Importo numpy para utilizar operaciones con el timedelta.

In [ ]:
import numpy as np

Calculo las duraciones de los usuarios haciendo fecha de bloqueo - fecha de alta en el sitio. Filtro con >0 a cero, dado que hay bugs en los datos. *Realice esta consulta dado que me daban resultados extraños, ya que habia numeros muy chicos y a su vez muy grandes, y en teoria todos deberian ser mayores a cero, dado que al trabajar con usuarios registrados, antes de ser bloqueados tendrian que ser registrados.*

In [ ]:
rddDelta = rddJoin.map(lambda x:(x[1][1]-x[1][0])).filter(lambda x: x>np.timedelta64(0))

Libero RAM.

In [ ]:
del rddJoin

**Resultado**

In [ ]:
rddDelta.min()

Timedelta('0 days 00:00:04')